# score_iot.py 붙여넣는 곳

 - 붙여넣은 후, init 함수 내에
  - edge_config = '/home/data/edge_config.yml' 부분을
  - edge_config = '../deployment/config/edge_config.yml' 로 수정하여 테스트

In [3]:
import sys
sys.path.append("../src")
from pkgs.mc_noisetest.function import preprocess
pr = preprocess()

/anaconda/envs/py37_op/lib/python3.7/site-packages/noisereduce/noisereduce.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


### 테스트를 위한 파라미터 입력

In [22]:
MODEL_VERSION = "03_051_01"
FILE_NAME = "data/pkl/{}.pickle".format(MODEL_VERSION)
EDGE_CONFIG = "config/edge_config.yml"
FEATURE_PARAMS_JSON = "../src/config/feature051_parameters.json"
COLUMN_NAMES_JSON = "../src/config/model_03_051_01_features.json"
def read_data(input_path, attempt):
    input_raw = pr.ReadDataBlob(input_path)
    return input_raw

### 테스트를 위해 아래의 PARAMETERS 영역 comment

In [23]:
# %%writefile src/score_iot.py


# For Edge
import json
import yaml
import time
import os
import re
import pickle
import shutil
from datetime import datetime
from datetime import timedelta
from time import sleep
from azureml.core.model import Model

# For Clf
import h5py
import pandas as pd
from classify import Classifier
from preprocess_signal_data import Run_
from pkgs.pipelines import FeatureDeployed as Featurizing
from pkgs.utils import read_json


# PARAMETERS --------------------------------------------------------
# MODEL_VERSION = "03_051_01"
# FILE_NAME = "{}.pickle".format(model_version)
# EDGE_CONFIG = "/home/data/edge_config.yml"
# FEATURE_PARAMS_JSON = "src/config/feature051_parameters.json"
# COLUMN_NAMES_JSON = "src/config/model_03_051_01_features.json"

# def read_data(input_path, attempt):
#     print("Attempt: " + str(attempt + 1) + "  Time: " + str(datetime.now()))
#     with h5py.File(input_path, "r") as f:
#         tmp = f["Raw"][:]
#     input_raw = pd.DataFrame(tmp, columns=[0, 1, 2])
#     print("\n", "hdf loaded")
    
#     return input_raw
# --------------------------------------------------------------------

def init_preprocessor(feature_params, Featurizing, test_no, input_type="ReadData"):
    # set preprocess class
    preproc = Run_(
        feature_params=feature_params, Featurizing=Featurizing, test_no=test_no
    )
    if input_type == "ReadData":
        preproc.pl.steps = [
            (step_name, step)
            for step_name, step in preproc.pl.steps
            if step.name != "ReadData"
        ]

    return preproc


def score(
    preproc,
    input_values,
    feature_names,
    column_names,
    cut_off,
    rule_model_params,
    mode,
    input_name="ReadData",
    parallel=1,
    verbose=0,
):
    # Preprocess
    df = preproc.pl.run(
        input_name=input_name,
        input_values=[input_values],
        feature_names=feature_names,
        parallel=parallel,
        verbose=verbose,
    )
    df = pd.DataFrame(df[0])
    df["TEST_NO"] = [0,1,2]
    df_features = df[column_names]

    # Predict
    df["PROB"] = loaded_model.predict_proba(df_features)[:,1]
    
    # Classify
    df["ML_R"] = df.PROB >= cut_off
    df["ML_R"] = df.ML_R.astype(int)
    df = Classifier.classify_all(
        df=df[df.TEST_NO.isin(preproc.featurizer.params["test_no"])],
        rule_cutoffs=rule_model_params,
        mode=mode,
    )
    return df, df_features


def init():
    global preproc, loaded_model, model_version, column_names, line, cut_off, rule_model_params, ml_model_params, model_path, error_dir
    # 수정필요  --------------------------------------------------------
    model_version = MODEL_VERSION  # model_name 입력
    file_name = FILE_NAME
    edge_config = EDGE_CONFIG
    feature_params = read_json(FEATURE_PARAMS_JSON)
    column_names = read_json(COLUMN_NAMES_JSON)["feature_names"]
    # ----------------------------------------------------------------

    # load line info
    with open(edge_config, "r") as stream:
        try:
            edge_config = yaml.load(stream, Loader=yaml.BaseLoader)
            line = edge_config["config"]["line"]["name"]
            cut_off = float(edge_config["config"]["param"]["cutoff"])
            rule_model_params = edge_config["config"]["param"]["rule_model"]
            ml_model_params = edge_config["config"]["param"]["ml_model"]

        except yaml.YAMLError as exc:
            print("line config error: ", exc)

    # set preprocess class
    for param_, dic_ in ml_model_params.items():
        for key_, value_ in dic_.items():
            feature_params[param_][key_] = float(value_)
    preproc = init_preprocessor(
        feature_params=feature_params, Featurizing=Featurizing, test_no=[0, 1, 2]
    )

    # load model
    model_path = Model.get_model_path(file_name)
    with open(model_path, "rb") as f:
        loaded_model = pickle.load(f)
    
    error_dir = "/home/data/error_file/"
    try:
        os.makedirs(error_dir, mode=777)
    except:
        pass


def run(input_json):
    print("\n", "mlmodule start")
    print("\n", datetime.now() + timedelta(hours=9), "\n")
    # for test#
    input_json = json.loads(input_json)
    print("\n", "json loaded", "\n")
    print(input_json, "\n")
    mltime = datetime.now() + timedelta(hours=9)
    chtime = input_json["chtime"]
    print("chtime : ", chtime)
    ct = datetime.strptime(
        chtime.replace("T", " ").split("+")[0][:-1], "%Y-%m-%d %H:%M:%S.%f"
    )
    diff = mltime - ct
    input_json["chtime"] = str(ct)
    input_json["mltime"] = str(mltime)
    input_json["etime_ch"] = diff.seconds + diff.microseconds / 1e6

    # file load
    init_time = time.time()
    # input_json = json.loads(input_json)
    input_path = input_json["path"]
    print("\n", input_path)

    for attempt in range(3):
        try:
            input_raw = read_data(input_path, attempt)

        except Exception as ex:
            if attempt == 2:
                ex_message = str(ex)

            sleep(0.02)
            continue

        break

    else:
        input_json["b"] = 2
        input_json["prob"] = 2
        input_json["error"] = ex_message
        input_json["etime"] = time.time() - init_time

        result_json = [json.dumps(input_json)]
        print("*" * 5, " ", "LOAD ERROR", " ", "*" * 5)
        print(result_json)
        try:
            shutil.copy(input_path, error_dir)
        except:
            pass

    # for test#
    # diff = (datetime.now()+ timedelta(hours=9))-mltime
    # input_json['etime_load'] = diff.seconds + diff.microseconds/1E6
    input_json["etime_load"] = time.time() - init_time

    ### json insert
    input_json["cutoff"] = cut_off
    input_json["TRHD_NM_SET"] = "/".join([key for key in rule_model_params.keys()] + [f"{key_}-{param}" for key_, dict_ in ml_model_params.items() for param, value in dict_.items()])
    input_json["TRHD_VAL_SET"] = "/".join(
        [str(value) for value in rule_model_params.values()]+[str(value) for key_, dict_ in ml_model_params.items() for param, value in dict_.items()]
    )
    input_json["FTUR_NM_SET"] = "/".join(
        [
            "/".join([col + "_TEST_N1" for col in column_names]),
            "/".join([col + "_TEST_N2" for col in column_names]),
            "/".join([col + "_TEST_N3" for col in column_names]),
        ]
    )
    input_json["lid"] = line
    input_json["v"] = model_version

    try:
        filename = input_path.split("/")[-1]
        filename = filename.strip("Data\\").split("_")
        input_json["bc"] = "_".join(filename[0:2])
        input_json["dtfull"] = filename[2][:-3]

    except Exception as ex:  # 에러 종류
        input_json["bc"] = "ERROR"
        input_json["dtfull"] = str(datetime.now())
        input_json["r"] = 2
        input_json["prob"] = 2
        input_json["error"] = str(ex)
        input_json["etime"] = time.time() - init_time

        result_json = [json.dumps(input_json)]
        print("*" * 5, " ", "FILENAME ERROR", " ", "*" * 5)
        print(result_json)
        try:
            cmd = "cp " + input_json["path"] + " " + error_dir
            os.system(cmd)
        except:
            pass

    try:
        df, df_features = score(
            preproc=preproc,
            input_values=input_raw,
            feature_names=preproc.featurizer.params["feature_names"],
            column_names=column_names,
            cut_off=cut_off,
            rule_model_params=rule_model_params,
            mode="or",
            input_name="ReadData",
            parallel=1,
            verbose=0,
        )
        
        input_json["r"] = int(df.R.values[0])
        input_json["prob"] = float(df.PROB.values[0])
        input_json["error"] = str(df.TEST_NG_R.values[0])
        input_json["etime"] = time.time() - init_time
        input_json["FTUR_VAL_SET"] = "/".join(
            [str(value) for value in df_features.values.flatten()]
        )

        result_json = [json.dumps(input_json)]
        print(result_json)

    except Exception as ex:  # 에러 종류
        input_json["r"] = 2
        input_json["prob"] = 2
        input_json["error"] = str(ex)
        input_json["etime"] = time.time() - init_time
        input_json["FTUR_VAL_SET"] = ""

        result_json = [json.dumps(input_json)]
        print("*" * 5, " ", "ERROR", " ", "*" * 5)
        print(result_json)
        try:
            cmd = "cp " + input_json["path"] + " " + error_dir
            os.system(cmd)
        except:
            pass

    return result_json


In [24]:
fpaths = ['/mc-origin-data-jyt/5/R/2020-03-04/P010400300B7B92TLR01_01336014700_2020-03-04-10-01-21.h5',
       '/mc-origin-data-jyt/5/R/2020-03-04/P010400300B7D9YCT701_01336014700_2020-03-04-10-06-32.h5',
       '/mc-origin-data-jyt/5/R/2020-03-04/P010400300B81CVWDP01_01336014700_2020-03-04-10-15-19.h5',
       '/mc-origin-data-jyt/5/R/2020-03-04/P010400300B892EBY901_01336014700_2020-03-04-10-43-25.h5',
       '/mc-origin-data-jyt/5/R/2020-03-04/P010400300D5A9A6TQ01_01336015000_2020-03-04-10-49-29.h5',
       '/mc-origin-data-jyt/5/R/2020-03-05/P01040030138CAHQ1O01_00013360173_2020-03-05-10-53-18.h5',
       '/mc-origin-data-jyt/5/R/2020-03-06/P01040030187EBO79101_01336014700_2020-03-06-11-16-55.h5',
       '/mc-origin-data-jyt/5/R/2020-03-06/P0104003018A60Z4HG01_01336014700_2020-03-06-14-04-48.h5',
       '/mc-origin-data-jyt/5/R/2020-03-06/P0104003018A8EZ4HG01_01336014700_2020-03-06-14-11-39.h5',
       '/mc-origin-data-jyt/5/R/2020-03-06/P0104003018AC2REGE01_01336014700_2020-03-06-14-19-30.h5',
       '/mc-origin-data-jyt/5/R/2020-03-09/P010400301DB2C4V2201_01336014700_2020-03-09-11-58-37.h5',
       '/mc-origin-data-jyt/5/R/2020-03-09/P010400301DCC64V2201_01336014700_2020-03-09-14-05-04.h5']

In [25]:
init()

In [30]:
csv_path = "../data/raw/1/R/2021-03-15/P010401303D479GHXE60_00013440047_2021-03-15-08-37-00.h5"
csv_path = "../data/raw/1/R/2021-03-15/P010401303D47A4KRV60_00013440047_2021-03-15-09-20-21.h5"
csv_path = "../data/raw/1/R/2021-03-15/P010401303D47BQOLZ60_00013440047_2021-03-15-09-25-45.h5"
# csv_path = '/mc-origin-data-jyt/1/R/2020-01-02/P01040010003113VXB01_00013450049_2020-01-02-10-07-32.h5'
# csv_path = "/mc-origin-data-jyt/1/R/2020-03-05/P0104003013058ZCV701_01345002300_2020-03-05-13-50-20.h5"
# csv_path = "/mc-origin-data-jyt/2/R/2020-03-05/P0104003010D8215KA01_00013470049_2020-03-05-09-36-11.h5"
# csv_path = fpaths[7]
chtime = "2020-03-12T16:47:04.7050953+09:00"
input_str = json.dumps({"path":csv_path,"chtime":chtime})
input_str 

'{"path": "../data/raw/1/R/2021-03-15/P010401303D47BQOLZ60_00013440047_2021-03-15-09-25-45.h5", "chtime": "2020-03-12T16:47:04.7050953+09:00"}'

In [31]:
run(input_str)


 mlmodule start

 2021-04-07 16:32:43.402013 


 json loaded 

{'path': '../data/raw/1/R/2021-03-15/P010401303D47BQOLZ60_00013440047_2021-03-15-09-25-45.h5', 'chtime': '2020-03-12T16:47:04.7050953+09:00'} 

chtime :  2020-03-12T16:47:04.7050953+09:00

 ../data/raw/1/R/2021-03-15/P010401303D47BQOLZ60_00013440047_2021-03-15-09-25-45.h5
Attempt: 1  Time: 2021-04-07 07:32:43.402638

 hdf loaded
['{"path": "../data/raw/1/R/2021-03-15/P010401303D47BQOLZ60_00013440047_2021-03-15-09-25-45.h5", "chtime": "2020-03-12 16:47:04.705095", "mltime": "2021-04-07 16:32:43.402460", "etime_ch": 85538.697365, "etime_load": 0.14560246467590332, "cutoff": 0.5, "TRHD_NM_SET": "FTUR_ENRG/FTUR_WVFM_STDDEV/FTUR_TRGER/FilterActiveNoiseCanceling-prop_decrease", "TRHD_VAL_SET": "0.00003/0.12/1/1.0", "FTUR_NM_SET": "FTUR_CPSR_MAX_N1_TEST_N1/FTUR_CPSR_MAX_N2_TEST_N1/FTUR_CPSR_MAX_N3_TEST_N1/FTUR_CPSR_MAX_N4_TEST_N1/FTUR_CPSR_MXRMS_N1_TEST_N1/FTUR_CPSR_MXRMS_N2_TEST_N1/FTUR_CPSR_MXRMS_N3_TEST_N1/FTUR_CPSR_MXRMS_N4_T

['{"path": "../data/raw/1/R/2021-03-15/P010401303D47BQOLZ60_00013440047_2021-03-15-09-25-45.h5", "chtime": "2020-03-12 16:47:04.705095", "mltime": "2021-04-07 16:32:43.402460", "etime_ch": 85538.697365, "etime_load": 0.14560246467590332, "cutoff": 0.5, "TRHD_NM_SET": "FTUR_ENRG/FTUR_WVFM_STDDEV/FTUR_TRGER/FilterActiveNoiseCanceling-prop_decrease", "TRHD_VAL_SET": "0.00003/0.12/1/1.0", "FTUR_NM_SET": "FTUR_CPSR_MAX_N1_TEST_N1/FTUR_CPSR_MAX_N2_TEST_N1/FTUR_CPSR_MAX_N3_TEST_N1/FTUR_CPSR_MAX_N4_TEST_N1/FTUR_CPSR_MXRMS_N1_TEST_N1/FTUR_CPSR_MXRMS_N2_TEST_N1/FTUR_CPSR_MXRMS_N3_TEST_N1/FTUR_CPSR_MXRMS_N4_TEST_N1/FTUR_RMS_TEST_N1/FTUR_SPTR_1000_3000_RMS_TEST_N1/FTUR_SPTR_3000_6000_RMS_TEST_N1/FTUR_SPTR_6000_9000_RMS_TEST_N1/FTUR_CPSR_MAX_N1_TEST_N2/FTUR_CPSR_MAX_N2_TEST_N2/FTUR_CPSR_MAX_N3_TEST_N2/FTUR_CPSR_MAX_N4_TEST_N2/FTUR_CPSR_MXRMS_N1_TEST_N2/FTUR_CPSR_MXRMS_N2_TEST_N2/FTUR_CPSR_MXRMS_N3_TEST_N2/FTUR_CPSR_MXRMS_N4_TEST_N2/FTUR_RMS_TEST_N2/FTUR_SPTR_1000_3000_RMS_TEST_N2/FTUR_SPTR_3000_600